In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForVision2Seq

tokenizer = AutoTokenizer.from_pretrained("emisilab/model-ocr-invoice-v1")
model = AutoModelForVision2Seq.from_pretrained("emisilab/model-ocr-invoice-v1")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

# Baixar a imagem
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# Carregar o processador e o modelo pré-treinados
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-handwritten')

# Processar a imagem
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Definir parâmetros de geração
generate_kwargs = {
    "max_length": 50,  # Ajuste conforme necessário
    "num_beams": 5,  # Ajuste conforme necessário
    "temperature": 0.7,  # Ajuste conforme necessário
}

# Passar a imagem pelo modelo
generated_ids = model.generate(pixel_values, **generate_kwargs)

# Decodificar os IDs gerados para texto
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Texto reconhecido:", generated_text)


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Texto reconhecido: 100 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000
